In [ ]:
import requests
from pymed import PubMed
import pickle

In [ ]:
def unique_authors(lst_authors):
    return [dicto['firstname']+ ' '+dicto['lastname'] for dicto in lst_authors if dicto['lastname'] and dicto['firstname']]

def unique_keywords(lst_keywords):
    pass

In [ ]:
pubmed = PubMed(tool="MyTool", email="my@email.address")
query = "covid[Title/Abstract] (2020[Date - Create] : 3000[Date - Create])"
# query formulation tool : https://pubmed.ncbi.nlm.nih.gov/advanced/
results = pubmed.query(query, max_results=100000)

dict_authors = {}
dict_keywords = {}

for article in results:
    try:
        authors = unique_authors(article.authors)
        keywords = article.keywords
        for author in authors:
            if author not in dict_authors.keys():
                dict_authors[author] = {}

        for keyword in keywords:
            keyword = keyword.lower()
            for author in authors:
                if keyword not in dict_authors[author]:
                    dict_authors[author][keyword] = 1
                else:
                    dict_authors[author][keyword] += 1

            if keyword not in dict_keywords:
                dict_keywords[keyword] = []
    except Exception as e:
        print(e)

In [ ]:
with open('keywords_dict_no_vectors.pickle', 'wb') as handle:
    pickle.dump(dict_keywords,handle)
with open('dict_authors.pickle', 'wb') as handle:
    pickle.dump(dict_authors,handle)


In [2]:
from transformers import *
import torch
import numpy as np

scibert_tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
scibert_model = AutoModel.from_pretrained('dmis-lab/biobert-v1.1')

In [5]:
counter=0
lst_keys = dict_keyword_no_vectors.keys()
dicto = {}
for key in lst_keys:
    counter+=1
    print(counter,end='\r')
    if dict_keyword_no_vectors[key] == []:
        try:
            token_keyword = torch.tensor(scibert_tokenizer.encode(key)).unsqueeze(0)
            out = scibert_model(token_keyword)
            dicto[key] = out[0][0][1:-1].detach().numpy().mean(axis=0)
        except ValueError:
            pass

<ipython-input-5-f04bae5c60a5>:11: RuntimeWarning: Mean of empty slice.
  dicto[key] = out[0][0][1:-1].detach().numpy().mean(axis=0)
/home/bastien/.local/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [ ]:
with open('keywords_and_vectors_v2.pkl','wb') as file:
    pickle.dump(dicto,file)